# Wie lernt eine KI?

Stell dir vor, du zeigst einem Kind Fotos von drei verschiedenen Blumenarten. Nach einer Weile kann das Kind neue Blumen richtig zuordnen, ohne sie vorher gesehen zu haben. Genau so funktioniert maschinelles Lernen!

Heute werden wir:
1. **Verstehen**, woher unsere Daten kommen
2. **Erkunden**, wie diese Daten aussehen
3. **Trainieren** eine KI, die Blumen unterscheiden kann
4. **Testen**, ob die KI wirklich gelernt hat

---

## Schritt 1: Die Geschichte unserer Daten

### Der Iris-Datensatz

Unsere Daten stammen aus dem Jahr **1936** Ein britischer Biologe namens **Ronald Fisher** hat drei Arten von Iris-Blumen untersucht:

- **Iris setosa** (Borsten-Schwertlilie)
- **Iris versicolor** (Verschiedenfarbige Schwertlilie)
- **Iris virginica** (Virginische Schwertlilie)

<img src="internal/iris_vergleich.jpg" alt="Iris Vergelich" width="800"/>

Er hat von jeder Art **50 Blumen** genommen und vier Dinge gemessen:

1. **Kelchblatt-Länge** (sepal length): Die langen grünen Blätter außen
2. **Kelchblatt-Breite** (sepal width): Wie breit diese Blätter sind
3. **Blütenblatt-Länge** (petal length): Die bunten Blätter innen
4. **Blütenblatt-Breite** (petal width): Wie breit diese sind

**Insgesamt: 150 Blumen × 4 Messungen = 600 Zahlen**

### Aufgabe 1: Den Datensatz verstehen

Beantworte folgende Fragen basierend auf dem Text oben:

1. **Wie viele verschiedene Eigenschaften** (Merkmale) wurden pro Blume gemessen?

2. Wenn du eine neue Blume findest und sie einer der drei Arten zuordnen willst - **welche vier Dinge müsstest du messen?**
---

## Schritt 2: Module vorbereiten

Bevor wir starten, brauchen wir zuerset die nötigen Python Module. Das sind fertige Programmteile, die andere Leute geschrieben haben.

**Was macht welches Modul?**
- `numpy`: Rechnen mit vielen Zahlen gleichzeitig
- `pandas`: Daten in Tabellen anzeigen
- `matplotlib`: Diagramme zeichnen
- `sklearn`: Die KI-Werkzeuge!

In [ ]:
import sys
import warnings

# Warnungen von threadpoolctl unterdrücken (Pyodide-Kompatibilitätsproblem)
warnings.filterwarnings("ignore", message=".*as_object_map.*")

# Installation der Module
if "pyodide" in sys.modules:
    import piplite

    print("⏳ Installiere Pakete für JupyterLite...")
    await piplite.install(["numpy", "pandas", "matplotlib", "scikit-learn"])
    print("✓ Fertig!")

# Import der Module
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

print("✓ Alle Module erfolgreich importiert!")

## Schritt 3: Daten laden und verstehen

Jetzt laden wir den Iris-Datensatz. Er befindet sich im Ordner `data` in der Datei iris.csv

![iris.csv](internal/csv_location.png)

Eine csv Datei ist ähnlich wie eine Excel Tabelle, mit dem Unterschied, dass man dabei nur Daten speichern kann, aber keine Grafiken oder eingebetteten Formeln.

In der ersten Zeile werden meistens die Spalten benannt:
- sepal_length
- sepal_width
- petal_length
- petal_width
- species

Das sind genau die Datenpunkte welche Ronald Fisher gemessen hat. Die letzte Spalte beinhaltet noch die Art.

Jetzt möchten wir mit Python die Date einlesen. Da csv ein sehr verbreitetes Format ist, geht das mit nur einer Zeile.

In [ ]:
# CSV-Datei laden
df = pd.read_csv("data/iris.csv")

Dabei steht `df` für DataFrame

Ein DataFrame ist wie eine Excel-Tabelle in Python. Es beinhaltet:
- Zeilen (horizontal) - jede Zeile ist ein Datensatz (z.B. eine Blume)
- Spalten (vertikal) - jede Spalte ist eine Eigenschaft (z.B. sepal_length, species)
- Spaltennamen - oben drüber stehen die Namen (sepal_length, sepal_width, etc.)

Mit einem DataFrame kannst du einfach mit Tabellendaten arbeiten: sortieren, filtern, rechnen, und vieles mehr.

### Die Daten als Tabelle anschauen

Zahlen in Listen sind schwer zu lesen. Schauen wir sie uns als Tabelle an!

In [ ]:
# Daten in eine schöne Tabelle verwandeln
print("Die ersten 10 Blumen in unserem Datensatz:\n")
print(df.head(10))

print("\nStatistik zu unseren Daten:")
print(df.describe())

### Aufgabe 2: Daten lesen und interpretieren

Schau dir die Tabelle oben genau an und beantworte diese Fragen:

1. **Finde die Blume in Zeile 5.** Wie lang ist ihr Blütenblatt (petal_length)?

2. **Vergleiche Zeile 0 und Zeile 4.** Welche Blume hat das breitere Kelchblatt (sepal_width)?

3. **Schau dir die Statistik an** (die zweite Tabelle mit mean, std, min, max):
   - Was ist die **durchschnittliche** Kelchblatt-Länge (sepal_length) aller 150 Blumen?
   - Was ist die **kleinste** gemessene Blütenblatt-Breite (petal_width)?

4. **Denke nach:** Die Standardabweichung (std) zeigt, wie stark die Werte streuen. Bei welchem Merkmal streuen die Werte am meisten? Was könnte das bedeuten?

---

### Was sehen wir hier?

Jede Zeile ist eine Blume. Die ersten vier Spalten sind die **Messungen** (in Zentimetern), die letzte Spalte ist die **Blumenart**.

**Beispiel Zeile 0:**
- Kelchblatt: 5.1 cm lang, 3.5 cm breit
- Blütenblatt: 1.4 cm lang, 0.2 cm breit
- → Das ist eine **Setosa**

Die KI soll lernen: "Wenn die Messungen so aussehen, dann ist es wahrscheinlich eine Setosa!"

## Schritt 4: Daten visualisieren

Ein Bild sagt mehr als tausend Zahlen! Schauen wir uns an, ob die Blumenarten wirklich unterschiedlich sind.

In [ ]:
# Streudiagramme: Kelchblatt vs Blütenblatt
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
colors = ["red", "green", "blue"]

for i, species in enumerate(df["species"].unique()):
    data = df[df["species"] == species]
    ax1.scatter(data["sepal_length"], data["sepal_width"], c=colors[i], label=species, s=60)
    ax2.scatter(data["petal_length"], data["petal_width"], c=colors[i], label=species, s=60)

ax1.set(xlabel="Kelchblatt-Länge (cm)", ylabel="Kelchblatt-Breite (cm)", title="Kelchblatt-Masse")
ax2.set(xlabel="Blütenblatt-Länge (cm)", ylabel="Blütenblatt-Breite (cm)", title="Blütenblatt-Masse")
ax1.legend()
ax2.legend()
plt.tight_layout()
plt.show()

### Warum ist das wichtig?

Schau dir das rechte Diagramm an (Blütenblatt). Die roten Punkte (Setosa) sind **weit weg** von den anderen. Das bedeutet: Setosa ist leicht zu erkennen!

Die grünen und blauen Punkte überlappen etwas. Das wird schwieriger für die KI.

### Aufgabe 3: Diagramme interpretieren

Schau dir die beiden Streudiagramme oben genau an und beantworte:

1. **Welche Blumenart** (Farbe) lässt sich am einfachsten von den anderen unterscheiden? Woran erkennst du das im Diagramm?

2. **Welche zwei Blumenarten** überlappen sich teilweise? In welchem der beiden Diagramme siehst du das deutlicher?

3. **Welches Diagramm** wäre besser geeignet, um die drei Arten zu unterscheiden - das linke (Kelchblatt) oder das rechte (Blütenblatt)? Begründe deine Antwort.

4. **Stell dir vor**, du bist die KI: Du siehst eine Blume mit Blütenblatt-Länge 5 cm und Blütenblatt-Breite 1.8 cm. Schau im rechten Diagramm nach - welche Art könnte das sein? Bist du dir sicher?

---

## Schritt 5: K-Means Clustering - Kann die KI Gruppen selbst finden?

### Zwei Arten zu lernen

Bevor wir der KI die richtigen Antworten geben, probieren wir etwas Spannendes: **Was passiert, wenn die KI KEINE Antworten bekommt?**

Stell dir vor:
- **Mit Antworten (überwachtes Lernen)**: Ein Lehrer zeigt dir Blumen und sagt: "Das ist eine Setosa, das ist eine Versicolor..."
- **Ohne Antworten (unüberwachtes Lernen)**: Du bekommst 150 Blumen und sollst sie selbst in Gruppen sortieren - ohne zu wissen, wie die Gruppen heissen!

Das zweite nennt man **Clustering** - die KI sucht selbstständig nach Mustern.

### Was ist K-Means?

K-Means ist ein einfacher aber cleverer Algorithmus:

1. **K** steht für die Anzahl Gruppen (bei uns: 3, weil wir 3 Blumenarten haben)
2. **Means** bedeutet "Durchschnitte" - jede Gruppe hat einen Mittelpunkt

**So funktioniert es:**

```
Schritt 1: Setze 3 zufällige Mittelpunkte (●) in die Daten
Schritt 2: Jede Blume gehört zum nächsten Mittelpunkt
Schritt 3: Berechne neue Mittelpunkte (Durchschnitt aller Blumen in der Gruppe)
Schritt 4: Wiederhole Schritt 2-3 bis sich nichts mehr ändert
```

Stell dir vor, du wirfst 3 Magnete in einen Haufen Büroklammern - die Klammern ordnen sich automatisch um die Magnete!

In [ ]:
# K-Means Clustering anwenden
X_cluster = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans.fit(X_cluster)
cluster_labels = kmeans.labels_

print("✅ K-Means hat 3 Gruppen gefunden:")
for i in range(3):
    print(f"   Gruppe {i}: {(cluster_labels == i).sum()} Blumen")

### K-Means Schritt für Schritt

Schauen wir uns an, wie K-Means die Gruppen findet. In jedem Schritt passiert folgendes:
1. Die **schwarzen X** sind die aktuellen Mittelpunkte
2. Jede Blume wird dem **nächsten Mittelpunkt** zugeordnet (Farbe)
3. Die Mittelpunkte werden **neu berechnet** (Durchschnitt aller Blumen in der Gruppe)

In [ ]:
# K-Means Iterationen visualisieren
X_vis = df[["petal_length", "petal_width"]].values
colors = ["purple", "orange", "cyan"]
titles = ["Iteration 1: Start", "Iteration 2", "Iteration 3", "Endergebnis"]

fig, axes = plt.subplots(2, 2, figsize=(10, 8))
np.random.seed(42)

for idx, (ax, max_iter) in enumerate(zip(axes.flatten(), [1, 2, 3, 10])):
    km = KMeans(n_clusters=3, max_iter=max_iter, n_init=1, random_state=42, init='random')
    km.fit(X_vis)
    
    for c in range(3):
        ax.scatter(X_vis[km.labels_ == c, 0], X_vis[km.labels_ == c, 1], c=colors[c], alpha=0.6, s=50)
    ax.scatter(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1], c="black", marker="X", s=200)
    ax.set(title=titles[idx], xlabel="Blütenblatt-Länge (cm)", ylabel="Blütenblatt-Breite (cm)")

plt.suptitle("K-Means lernt Schritt für Schritt", fontweight="bold")
plt.tight_layout()
plt.show()

### K-Means Ergebnis visualisieren

Schauen wir uns an, wie K-Means die Blumen gruppiert hat - und vergleichen es mit den echten Blumenarten!

In [ ]:
# Vergleich: K-Means Cluster vs. echte Blumenarten
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
cluster_colors = ["purple", "orange", "cyan"]
real_colors = ["red", "green", "blue"]

# K-Means Cluster
for c in range(3):
    mask = cluster_labels == c
    ax1.scatter(df.loc[mask, "petal_length"], df.loc[mask, "petal_width"], c=cluster_colors[c], label=f"Cluster {c}", s=60)
ax1.scatter(kmeans.cluster_centers_[:, 2], kmeans.cluster_centers_[:, 3], c="black", marker="X", s=150)
ax1.set(xlabel="Blütenblatt-Länge (cm)", ylabel="Blütenblatt-Breite (cm)", title="K-Means Cluster")
ax1.legend()

# Echte Blumenarten
for i, species in enumerate(df["species"].unique()):
    data = df[df["species"] == species]
    ax2.scatter(data["petal_length"], data["petal_width"], c=real_colors[i], label=species, s=60)
ax2.set(xlabel="Blütenblatt-Länge (cm)", ylabel="Blütenblatt-Breite (cm)", title="Echte Blumenarten")
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Wie gut passt K-Means zu den echten Blumenarten?
print("📊 Zuordnung der Cluster zu den echten Blumenarten:\n")

for cluster in range(3):
    print(f"Cluster {cluster}:", end="  ")
    for species in df["species"].unique():
        count = ((cluster_labels == cluster) & (df["species"] == species)).sum()
        if count > 0:
            print(f"{species}: {count}", end="  ")
    print()

### Aufgabe 4: K-Means verstehen

Analysiere die K-Means Ergebnisse und beantworte:

1. **Vergleiche die beiden Diagramme oben.** Hat K-Means die gleichen Gruppen gefunden wie die echten Blumenarten? Beschreibe die Ähnlichkeiten und Unterschiede.

2. **Schau dir die Zuordnungstabelle an.** Welche Blumenart wurde am "saubersten" erkannt (fast alle in einem Cluster)? Welche wurde auf mehrere Cluster verteilt?

3. **Gedankenexperiment:** K-Means wusste NICHT, dass es drei Blumenarten gibt - wir haben ihm nur gesagt "finde 3 Gruppen". Was wäre passiert, wenn wir `n_clusters=2` oder `n_clusters=5` gewählt hätten?

4. **Kritisches Denken:** In welchen Situationen wäre unüberwachtes Lernen (wie K-Means) nützlicher als überwachtes Lernen? Denke an ein Beispiel, wo man die "richtigen Antworten" noch nicht kennt.

---

---

## Von unüberwacht zu überwacht

Wir haben gesehen: K-Means kann Gruppen finden, **ohne die Antworten zu kennen**. Das ist beeindruckend!

Aber jetzt kommt der Clou: **Wir HABEN ja die richtigen Antworten!** Wir wissen, welche Blume zu welcher Art gehört.

Warum sollten wir dieses Wissen nicht nutzen?

| Methode | Vorteile | Nachteile |
|---------|----------|-----------|
| **K-Means (unüberwacht)** | Braucht keine Labels | Kennt die echten Kategorien nicht |
| **Neuronales Netz (überwacht)** | Lernt aus Beispielen mit Antworten | Braucht gelabelte Daten |

**Jetzt trainieren wir ein neuronales Netz** - eine KI, die aus Beispielen MIT den richtigen Antworten lernt!

---

## Schritt 6: Daten aufteilen

### Das Schul-Prinzip

Stell dir vor, du lernst für eine Prüfung:
- **Übungsaufgaben** → damit lernst du (= Trainingsdaten)
- **Prüfungsaufgaben** → damit wird getestet, ob du's verstanden hast (= Testdaten)

Wenn die Prüfung die gleichen Aufgaben hat wie beim Üben, wüssten wir nicht, ob du wirklich verstanden hast oder nur auswendig gelernt hast!

**Deshalb teilen wir unsere 150 Blumen auf:**
- 80% (120 Blumen) → Training
- 20% (30 Blumen) → Test

### Aufgabe 5: Warum Daten aufteilen?

Denke über das "Schul-Prinzip" nach:

1. **Analogie verstehen:** Erkläre in deinen eigenen Worten, warum es unfair wäre, wenn eine Prüfung genau die gleichen Aufgaben hätte wie die Übungen.

2. **Übertragung auf KI:** Was würde passieren, wenn wir die KI mit ALLEN 150 Blumen trainieren und dann mit den gleichen 150 Blumen testen? Wäre das ein guter Test?

3. **Der Begriff "Overfitting":** Wenn eine KI die Trainingsdaten "auswendig lernt" statt Muster zu erkennen, nennt man das Overfitting. Erkläre mit einem Beispiel aus der Schule, was das bedeuten könnte.

4. **Rechenaufgabe:** Wir haben 150 Blumen. 80% davon sind Trainingsdaten. Wie viele Blumen sind das? Wie viele bleiben für den Test übrig?

---

In [ ]:
# Daten aufteilen: 80% Training, 20% Test
X = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
y = df["species"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"📚 Trainingsdaten: {len(X_train)} Blumen | Testdaten: {len(X_test)} Blumen")

## Schritt 7: Normalisierung - Alle auf eine Skala bringen

### Das Problem

Stell dir vor, du vergleichst:
- Die Geschwindigkeit eines Autos: 120 km/h
- Das Gewicht eines Autos: 1500 kg

Die Zahlen sind sehr unterschiedlich gross! Die KI könnte denken, dass das Gewicht wichtiger ist, nur weil die Zahl grösser ist.

### Die Lösung: Normalisierung

Wir verwandeln alle Werte so, dass sie ungefähr zwischen -2 und +2 liegen. Dann sind alle Merkmale gleich wichtig.

In [ ]:
# Normalisierung: Alle Werte auf ähnliche Skala bringen
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Vorher:  {X_train.iloc[0].values}")
print(f"Nachher: {X_train_scaled[0].round(2)}")

## Schritt 8: Das neuronale Netz erstellen

### Was ist ein neuronales Netz?

Ein neuronales Netz ist wie ein Team von Entscheidungshelfern:

```
Eingabe (4 Zahlen)    Versteckte Schicht    Ausgabe (3 Zahlen)
                      
Kelchblatt-Länge ──┐  
Kelchblatt-Breite ─┼──→ [10 Helfer] ──┐
Blütenblatt-Länge ─┼──→ [10 Helfer] ──┼──→ Setosa?
Blütenblatt-Breite ┘                   ├──→ Versicolor?
                                       └──→ Virginica?
```

- **Eingabe**: Die 4 Messungen
- **Versteckte Schichten**: Hier passiert das "Denken"
- **Ausgabe**: Wahrscheinlichkeit für jede Blumenart

In [ ]:
# Neuronales Netz erstellen
model = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=42)

print("🧠 Netz erstellt: [4 Eingänge] → [10 Neuronen] → [10 Neuronen] → [3 Ausgänge]")

## Schritt 9: Training - Jetzt lernt die KI!

### Was passiert beim Training?

1. Die KI schaut sich eine Blume an
2. Sie rät, welche Art es ist
3. Wir sagen ihr, ob sie richtig lag
4. Sie passt ihre inneren "Verbindungen" an
5. Beim nächsten Mal rät sie hoffentlich besser

Das macht sie 1000 Mal mit allen 120 Trainingsblumen!

In [ ]:
# Training durchführen
model.fit(X_train_scaled, y_train)
print("✅ Training abgeschlossen!")

## Schritt 10: Die grosse Prüfung!

Jetzt zeigen wir der KI die 30 Testblumen, die sie noch NIE gesehen hat.

In [ ]:
# Vorhersagen und Genauigkeit
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print(f"🎯 Ergebnis: {(y_test == y_pred).sum()}/{len(y_test)} richtig = {accuracy*100:.1f}% Genauigkeit")

### Einzelne Vorhersagen anschauen

In [ ]:
# Einzelne Vorhersagen anschauen
print("🔍 Die ersten 10 Vorhersagen:\n")
for i in range(min(10, len(y_test))):
    actual, predicted = y_test.iloc[i], y_pred[i]
    status = "✅" if actual == predicted else "❌"
    print(f"{status} Blume {i+1}: {actual} → {'Richtig!' if actual == predicted else f'Fehler ({predicted})'}")

### Aufgabe 6: Die Confusion Matrix lesen

Analysiere die Confusion Matrix oben und beantworte:

1. **Grundverständnis:** Die Diagonale zeigt richtige Vorhersagen. Zähle zusammen: Wie viele Blumen wurden insgesamt RICHTIG erkannt?

2. **Fehler finden:** Gibt es Felder NEBEN der Diagonale mit Zahlen grösser als 0? Wenn ja, welche Blumenarten wurden verwechselt?

3. **Muster erkennen:** Erinnere dich an die Streudiagramme von Aufgabe 3. Welche Blumenarten haben sich dort überlappt? Passt das zu den Verwechslungen in der Confusion Matrix?

4. **Genauigkeit berechnen:** Die Genauigkeit (Accuracy) = Richtige / Gesamt × 100%. Berechne die Genauigkeit selbst anhand der Matrix. Stimmt dein Ergebnis mit dem angezeigten Ergebnis überein?

5. **Kritische Frage:** Stell dir vor, eine Blumenart wäre viel häufiger als die anderen (z.B. 140 Setosa, 5 Versicolor, 5 Virginica). Warum wäre die Accuracy dann kein gutes Mass mehr?

---

## Schritt 11: Wo macht die KI Fehler?

Die **Confusion Matrix** (Verwirrungsmatrix) zeigt uns, welche Arten die KI verwechselt.

In [ ]:
# Confusion Matrix erstellen und visualisieren
cm = confusion_matrix(y_test, y_pred)
species_names = df["species"].unique()

fig, ax = plt.subplots(figsize=(7, 5))
im = ax.imshow(cm, cmap="Blues")

ax.set(xticks=range(3), yticks=range(3), xticklabels=species_names, yticklabels=species_names,
       xlabel="Vorhergesagt", ylabel="Tatsächlich", title="Confusion Matrix")

for i in range(3):
    for j in range(3):
        ax.text(j, i, cm[i, j], ha="center", va="center", 
                color="white" if cm[i, j] > cm.max()/2 else "black", fontsize=16, fontweight="bold")

plt.colorbar(im)
plt.tight_layout()
plt.show()

print("💡 Diagonale = richtig erkannt, daneben = Fehler")

## Schritt 12: Teste die KI mit einer neuen Blume!

Jetzt kommt der spannende Teil: Wir "erfinden" eine Blume und schauen, was die KI sagt.

In [ ]:
# Teste die KI mit einer neuen Blume
# Format: [Kelchblatt-Länge, Kelchblatt-Breite, Blütenblatt-Länge, Blütenblatt-Breite]
neue_blume = np.array([[5.1, 3.5, 1.4, 0.2]])

neue_blume_scaled = scaler.transform(neue_blume)
vorhersage = model.predict(neue_blume_scaled)
proba = model.predict_proba(neue_blume_scaled)[0]

print(f"📏 Messungen: Kelchblatt {neue_blume[0][0]}×{neue_blume[0][1]} cm, Blütenblatt {neue_blume[0][2]}×{neue_blume[0][3]} cm")
print(f"\n🤖 Vorhersage: {vorhersage[0]}")
print(f"\n📊 Wahrscheinlichkeiten:")
for species, p in zip(df["species"].unique(), proba):
    print(f"   {species}: {'█' * int(p*20)} {p*100:.1f}%")

### 🎮 Experimentiere selbst!

Ändere die Werte in der Zeile `neue_blume = np.array([[...]])` und führe die Zelle nochmal aus!

**Beispiele zum Ausprobieren:**
```python
# Typische Setosa (kleine Blütenblätter):
neue_blume = np.array([[5.0, 3.5, 1.3, 0.3]])

# Typische Versicolor (mittlere Grösse):
neue_blume = np.array([[6.0, 2.7, 4.5, 1.5]])

# Typische Virginica (grosse Blütenblätter):
neue_blume = np.array([[6.5, 3.0, 5.5, 2.0]])

# Extreme Werte (was passiert?):
neue_blume = np.array([[8.0, 4.0, 7.0, 3.0]])
```

### 💡 Wichtige Konzepte:

| Konzept | Was bedeutet das? |
|---------|-------------------|
| **Unüberwachtes Lernen** | KI sucht Muster ohne die Antworten zu kennen (z.B. K-Means) |
| **Überwachtes Lernen** | KI lernt aus Beispielen MIT den richtigen Antworten |
| **Clustering** | Daten automatisch in Gruppen einteilen |
| **Training** | Die KI lernt aus Beispielen |
| **Test** | Überprüfung mit neuen, ungesehenen Daten |
| **Features** | Die Merkmale (hier: 4 Messungen) |
| **Labels** | Die richtigen Antworten (hier: Blumenarten) |
| **Overfitting** | Wenn die KI auswendig lernt statt Muster zu erkennen |
| **Accuracy** | Wie oft liegt die KI richtig? |

---

### 🚀 Nächste Schritte:

**Experimentiere weiter:**
- Ändere die Netzwerk-Grösse: `hidden_layer_sizes=(5, 5)` oder `(20, 20, 20)`
- Ändere die Lernrunden: `max_iter=500` oder `max_iter=2000`
- Was passiert mit nur 50% Trainingsdaten? Ändere: `test_size=0.5`
- Probiere K-Means mit anderen Cluster-Anzahlen: `n_clusters=2` oder `n_clusters=5`

---

### 🎉 Glückwunsch!

Du hast gerade:
- ✅ Einen echten Datensatz verstanden
- ✅ Daten für KI vorbereitet
- ✅ **K-Means Clustering** ausprobiert (unüberwachtes Lernen)
- ✅ Ein **neuronales Netz** trainiert (überwachtes Lernen)
- ✅ Die Qualität getestet
- ✅ Beide Methoden verglichen

**Das ist genau der Prozess, den auch professionelle Data Scientists verwenden!**

---

### 📚 Ressourcen zum Weiterlernen:

- **Scikit-learn Dokumentation**: https://scikit-learn.org/
- **Kaggle Learn**: Kostenlose Kurse zu Machine Learning
- **Fast.ai**: Praktische Deep Learning Kurse
- **3Blue1Brown**: Visuelle Erklärungen zu neuronalen Netzen (YouTube)

---

**💪 Viel Erfolg auf deiner KI-Reise!**